In [1]:
import cv2

In [2]:
import numpy as np

In [3]:
import os

In [4]:
import string

In [5]:
class Region:
    def __init__(self, label):
        self.label = label
        self.pixels = []
        self.area = 0
        self.min_x = 99999
        self.max_x = 0
        self.min_y = 99999
        self.max_y = 0
        self.cx = 0
        self.cy = 0
        self.perimeter = 0
        self.compactness = 0
        self.aspect_ratio = 0
        self.symbol = ""
    
    def finalize(self):
        self.area = len(self.pixels)
        sx = 0
        sy = 0
        for (x,y) in self.pixels:
            sx += x
            sy += y
        if self.area > 0:
            self.cx = sx / self.area
            self.cy = sy / self.area
        w = (self.max_x - self.min_x) + 1
        h = (self.max_y - self.min_y) + 1
        if h != 0:
            self.aspect_ratio = w / h
        else:
            self.aspect_ratio = 0
        if self.area > 0:
            self.compactness = (self.perimeter * self.perimeter) / self.area
        else:
            self.compactness = 0

In [6]:
def threshold_image(img):
    g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(g, (7,7), 0)
    _, bw = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    inv = 255 - bw
    kernel = np.ones((3,3), np.uint8)
    inv = cv2.morphologyEx(inv, cv2.MORPH_OPEN, kernel, iterations=2)
    inv = cv2.morphologyEx(inv, cv2.MORPH_CLOSE, kernel, iterations=2)
    return inv

In [7]:
def label_regions(binary_img, min_area=100):
    h, w = binary_img.shape
    visited = np.zeros((h,w), dtype=np.uint8)
    label = 1
    regions = {}
    
    for y in range(h):
        for x in range(w):
            if binary_img[y,x] == 255 and visited[y,x] == 0:
                temp_region = Region(label)
                stack = [(x,y)]
                visited[y,x] = 1
                
                while stack:
                    px,py = stack.pop()
                    temp_region.pixels.append((px,py))
                    if px < temp_region.min_x:
                        temp_region.min_x = px
                    if px > temp_region.max_x:
                        temp_region.max_x = px
                    if py < temp_region.min_y:
                        temp_region.min_y = py
                    if py > temp_region.max_y:
                        temp_region.max_y = py
                    
                    nb = [(px-1,py),(px+1,py),(px,py-1),(px,py+1),
                          (px-1,py-1),(px+1,py-1),(px-1,py+1),(px+1,py+1)]
                    for nx,ny in nb:
                        if nx>=0 and nx<w and ny>=0 and ny<h:
                            if binary_img[ny,nx]==255 and visited[ny,nx]==0:
                                visited[ny,nx]=1
                                stack.append((nx,ny))
                
                if len(temp_region.pixels) >= min_area:
                    regions[label] = temp_region
                    label += 1
    
    return regions

In [8]:
def compute_perimeters(binary_img, regions):
    h, w = binary_img.shape
    for label, region in regions.items():
        p = 0
        for x,y in region.pixels:
            for nx,ny in [(x-1,y),(x+1,y),(x,y-1),(x,y+1)]:
                if nx>=w or ny>=h or nx<0 or ny<0:
                    p+=1
                else:
                    if binary_img[ny,nx]==0:
                        p+=1
        region.perimeter = p
        region.finalize()

In [9]:
def classify_symbol(region):
    if region.compactness < 20:
        return "Square"
    elif region.compactness > 50:
        return "Circle"
    elif region.aspect_ratio > 1.4 or region.aspect_ratio < 0.7:
        return "Rectangle"
    else:
        return "Block"

In [10]:
def assign_symbols(regions):
    for label, r in regions.items():
        r.symbol = classify_symbol(r)

In [11]:
def train_model(train_images):
    training_data = []
    print("TRAINING PHASE")
    
    for imgname in train_images:
        if not os.path.exists(imgname):
            print(f"Missing: {imgname}")
            continue
        
        print(f"\n{imgname}:")
        img = cv2.imread(imgname)
        bw = threshold_image(img)
        regions = label_regions(bw, min_area=100)
        compute_perimeters(bw, regions)
        assign_symbols(regions)
        
        for label, r in regions.items():
            print(f"  Region {label}: Symbol={r.symbol}, Area={r.area}, Perimeter={r.perimeter}, "
                  f"AspectRatio={r.aspect_ratio:.2f}, Compactness={r.compactness:.2f}, "
                  f"Center=({r.cx:.2f}, {r.cy:.2f})")
            training_data.append({
                "symbol": r.symbol,
                "area": r.area,
                "compactness": r.compactness,
                "aspect_ratio": r.aspect_ratio
            })
    
    return training_data

In [13]:
def predict_symbols(img_name, training_data):
    img = cv2.imread(img_name)
    bw = threshold_image(img)
    regions = label_regions(bw, min_area=100)
    compute_perimeters(bw, regions)
    assign_symbols(regions)
    
    predicted = []
    for label, r in regions.items():
        best = None
        best_score = 999999
        for entry in training_data:
            da = abs(r.area - entry["area"])
            dc = abs(r.compactness - entry["compactness"])
            dr = abs(r.aspect_ratio - entry["aspect_ratio"])
            S = da*0.001 + dc*1 + dr*10
            if S < best_score:
                best_score = S
                best = entry["symbol"]
        
        predicted.append({
            "label": label,
            "symbol": best,
            "cx": r.cx,
            "cy": r.cy,
            "area": r.area,
            "perimeter": r.perimeter,
            "aspect_ratio": r.aspect_ratio,
            "compactness": r.compactness,
            "bbox": (r.min_x, r.min_y, r.max_x - r.min_x + 1, r.max_y - r.min_y + 1)
        })
    return predicted

In [14]:
def analyze_image(img_name, training_data):
    predictions = predict_symbols(img_name, training_data)
    predictions = sorted(predictions, key=lambda p: (p["cy"], p["cx"]))
    
    print(f"\n{img_name}:")
    labels = list(string.ascii_uppercase)
    for i, pred in enumerate(predictions):
        label = labels[i] if i < len(labels) else str(i)
        print(f"  Block {label}: Symbol={pred['symbol']}, Area={pred['area']}, Perimeter={pred['perimeter']}, "
              f"AspectRatio={pred['aspect_ratio']:.2f}, Compactness={pred['compactness']:.2f}, "
              f"Center=({pred['cx']:.2f}, {pred['cy']:.2f})")

In [15]:
def run_system():
    train_imgs = [f"image{i}.png" for i in range(1, 7)]
    test_imgs = [f"image{i}.png" for i in range(7, 11)]
    
    training_data = train_model(train_imgs)
    
    print("TESTING PHASE")
    
    for img in test_imgs:
        if os.path.exists(img):
            analyze_image(img, training_data)
        else:
            print(f"\nMissing: {img}")

run_system()

TRAINING PHASE

image1.png:
  Region 1: Symbol=Circle, Area=17984, Perimeter=1688, AspectRatio=0.32, Compactness=158.44, Center=(175.81, 180.57)

image2.png:
  Region 1: Symbol=Circle, Area=38651, Perimeter=1920, AspectRatio=0.98, Compactness=95.38, Center=(162.89, 156.71)

image3.png:
  Region 1: Symbol=Block, Area=56942, Perimeter=1342, AspectRatio=1.08, Compactness=31.63, Center=(164.76, 165.33)

image4.png:
  Region 1: Symbol=Circle, Area=117052, Perimeter=2688, AspectRatio=1.86, Compactness=61.73, Center=(360.09, 233.87)

image5.png:
  Region 1: Symbol=Block, Area=55738, Perimeter=1224, AspectRatio=1.39, Compactness=26.88, Center=(200.15, 139.30)

image6.png:
  Region 1: Symbol=Circle, Area=301, Perimeter=126, AspectRatio=8.00, Compactness=52.74, Center=(123.79, 31.88)
  Region 2: Symbol=Circle, Area=26894, Perimeter=1288, AspectRatio=1.41, Compactness=61.68, Center=(184.16, 177.80)
TESTING PHASE

image7.png:
  Block A: Symbol=Circle, Area=23445, Perimeter=1294, AspectRatio=1.98, 